**_pySpark Basics: Pivoting Data_**

_by Jeff Levy (jlevy@urban.org)_

_Last Updated: 2 Aug 2016, Spark v1.6.1_

_Abstract: This guide will illustrate how to pivot data._

_Main operations used: groupBy, pivot, sum_

***

# Reshaping

Pivoting involves an aggregation, as we will see below.  If what you're looking for is reshaping, where a dataset is turned from wide to long or vice versa without the loss of any information, then that is not currently implemented in Spark.  The likely reason for this is that it's an incredibly costly operation; if you've performed it in Stata or SAS, for example, it probably took a while to compute even on smaller datasets.  Doing it on very large data that is distributed across many nodes is even worse.
 
This may be something we see Spark implement with relative efficiency down the road, but for now the only way to accomplish it is with a custom looping structure that rebuilds the dataframe the way you want it.  In general however, if your data is very large, it's best to think about ways to avoid having to completely restructure it.

As we will show at the end however, it is possible to make `pivot` work similarly to reshaping in some cases.

In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Pivoting

To illustrate how pivoting works, we create a toy dataset:

In [2]:
from pyspark.sql import Row

row = Row('state', 'industry', 'hq', 'jobs')

df = sc.parallelize([
    row('MI', 'auto', 'domestic', 716),
    row('MI', 'auto', 'foreign', 123),
    row('MI', 'auto', 'domestic', 1340),
    row('MI', 'retail', 'foreign', 12),
    row('MI', 'retail', 'foreign', 33),
    row('OH', 'auto', 'domestic', 349),
    row('OH', 'auto', 'foreign', 101),
    row('OH', 'auto', 'foreign', 77),
    row('OH', 'retail', 'domestic', 45),
    row('OH', 'retail', 'foreign', 12)
    ]).toDF()

In [3]:
df.show()

+-----+--------+--------+----+
|state|industry|      hq|jobs|
+-----+--------+--------+----+
|   MI|    auto|domestic| 716|
|   MI|    auto| foreign| 123|
|   MI|    auto|domestic|1340|
|   MI|  retail| foreign|  12|
|   MI|  retail| foreign|  33|
|   OH|    auto|domestic| 349|
|   OH|    auto| foreign| 101|
|   OH|    auto| foreign|  77|
|   OH|  retail|domestic|  45|
|   OH|  retail| foreign|  12|
+-----+--------+--------+----+



Pivot operations must always be preceeded by a groupBy operation.  In our first case we will simply pivot to show domestic versus foreign jobs in each of our two states:

In [8]:
df_pivot1 = df.groupby('state').pivot('hq', values=['domestic', 'foreign']).sum('jobs')

In [9]:
df_pivot1.show()

+-----+--------+-------+
|state|domestic|foreign|
+-----+--------+-------+
|   OH|     394|    190|
|   MI|    2056|    168|
+-----+--------+-------+



**Note that the `values=['domestic', 'foreign']` part of the pivot method is optional.**  If we don't supply a list then pySpark will attempt to infer the values, but naturally that requires more processing than if we specify it up front.  As your datasets get larger and larger this sort of help becomes more and more important.

Here's another example, this time pivoting by both `state` and by `industry`:

In [15]:
df_pivot = df.groupBy('state', 'industry').pivot('hq', values=['domestic', 'foreign']).sum('jobs')

In [16]:
df_pivot.show()

+-----+--------+--------+-------+
|state|industry|domestic|foreign|
+-----+--------+--------+-------+
|   MI|    auto|    2056|    123|
|   OH|  retail|      45|     12|
|   OH|    auto|     349|    178|
|   MI|  retail|    null|     45|
+-----+--------+--------+-------+



The `sum` method at the end can be replaced as necessary, for example with `avg`.

# Approximating Reshape

Pivot requires an aggregation argument at the end, as we have been using.  However, what if each row is uniquely defined by the `groupby` and `pivot` columns?

In [10]:
row = Row('state', 'industry', 'hq', 'jobs', 'firm')

df = sc.parallelize([
    row('MI', 'auto', 'domestic', 716, 'A'),
    row('MI', 'auto', 'foreign', 123, 'B'),
    row('MI', 'auto', 'domestic', 1340, 'C'),
    row('MI', 'retail', 'foreign', 12, 'D'),
    row('MI', 'retail', 'foreign', 33, 'E'),
    row('OH', 'auto', 'domestic', 349, 'F'),
    row('OH', 'auto', 'foreign', 101, 'G'),
    row('OH', 'auto', 'foreign', 77, 'H'),
    row('OH', 'retail', 'domestic', 45, 'I'),
    row('OH', 'retail', 'foreign', 12, 'J')
    ]).toDF()

In [11]:
df.show()

+-----+--------+--------+----+----+
|state|industry|      hq|jobs|firm|
+-----+--------+--------+----+----+
|   MI|    auto|domestic| 716|   A|
|   MI|    auto| foreign| 123|   B|
|   MI|    auto|domestic|1340|   C|
|   MI|  retail| foreign|  12|   D|
|   MI|  retail| foreign|  33|   E|
|   OH|    auto|domestic| 349|   F|
|   OH|    auto| foreign| 101|   G|
|   OH|    auto| foreign|  77|   H|
|   OH|  retail|domestic|  45|   I|
|   OH|  retail| foreign|  12|   J|
+-----+--------+--------+----+----+



We've now added a unique identifier for each firm, which we will use instead of state and industry as our groupby criteria:

In [18]:
df_pivot = df.groupBy('firm').pivot('hq', values=['domestic', 'foreign']).sum('jobs')

In [19]:
df_pivot.show()

+----+--------+-------+
|firm|domestic|foreign|
+----+--------+-------+
|   A|     716|   null|
|   B|    null|    123|
|   C|    1340|   null|
|   D|    null|     12|
|   E|    null|     33|
|   F|     349|   null|
|   G|    null|    101|
|   H|    null|     77|
|   I|      45|   null|
|   J|    null|     12|
+----+--------+-------+



It dropped the columns we didn't use anywhere, state and industry, but if we wanted to keep them we can just include them the groupby criteria without changing the logic of the operation:

In [20]:
df_pivot = df.groupBy('firm', 'state', 'industry').pivot('hq', values=['domestic', 'foreign']).sum('jobs')

In [21]:
df_pivot.show()

+----+-----+--------+--------+-------+
|firm|state|industry|domestic|foreign|
+----+-----+--------+--------+-------+
|   D|   MI|  retail|    null|     12|
|   G|   OH|    auto|    null|    101|
|   F|   OH|    auto|     349|   null|
|   C|   MI|    auto|    1340|   null|
|   J|   OH|  retail|    null|     12|
|   B|   MI|    auto|    null|    123|
|   I|   OH|  retail|      45|   null|
|   A|   MI|    auto|     716|   null|
|   E|   MI|  retail|    null|     33|
|   H|   OH|    auto|    null|     77|
+----+-----+--------+--------+-------+



All we're doing is still telling it to `sum` each grouping of values, but **each grouping only has a single entry.**  Our data is now reshaped from long to wide, although it may not look "wide" just because there are only two entries in our pivot column, `hq`.  If we replaced the `sum` operator with `mean` or `max` or `min`, it wouldn't change anything.